# Data Indexing and Selection

ここでは、`Daru::Vector`や`Daru::DataFrame`に格納された値を取得したり、それらの値を修正する方法を解説します。

まずは、シンプルな一次元の`Vector`について説明します。その次に、もっと複雑な二次元の`DataFrame`について説明します。

## Vector as Hash

前章で見たように、`Vector`は一次元の`Array`のようにも、`Hash`のようにも扱えます。

### Vector as Hash

`Hash`と同様に、`Vector`は、`key`から`value`への対応付けを提供します。

In [1]:
require 'daru'

vector = Daru::Vector.new(
  [0.25, 0.5, 0.75, 1.0],
  index: [:a, :b, :c, :d]
)

#<Daru::Vector(4)>
    a 0.25
    b  0.5
    c 0.75
    d  1.0

In [2]:
vector[:b]

0.5

In [3]:
# vector.key? :b

In [4]:
vector.index

#<Daru::Index(4): {a, b, c, d}>

In [5]:
vector.to_h

{:a=>0.25, :b=>0.5, :c=>0.75, :d=>1.0}

In [6]:
# vector[:new_key] = 1.25

### Vector as one-dimensional Array

以下で示すとおり、組み込みの`Array`オブジェクトによく似た方法で、値を選択することができます。

In [7]:
# slicing by explicit index
vector[:a..:c]

#<Daru::Vector(3)>
    a 0.25
    b  0.5
    c 0.75

In [8]:
# slicing by implicit integer index
# 将来なくなるかも(https://github.com/v0dro/daru/issues/283)
vector[0..2]

#<Daru::Vector(3)>
    a 0.25
    b  0.5
    c 0.75

In [93]:
# masking
# 組み込みのArrayにこういう機能はない（強いて言うならArray#select？）。ちょっと変な感じ。
vector.where((vector.gt 0.3) & (vector.lt 0.8))

#<Daru::Vector(2)>
    b  0.5
    c 0.75

In [10]:
# fancy index
vector[:a, :d]

#<Daru::Vector(2)>
    a 0.25
    d  1.0

## Data Selection in DataFrame

### DataFrame as a Hash

１つめのアナロジーとして、`DataFrame`は`Vector`の`Hash`だと考えてみましょう：

In [11]:
area = Daru::Vector.new(
  {
    California: 423967, Texas: 695662,
    NewYork: 141297, Florida: 170312,
    Illinois: 149995
  }
)
pop = Daru::Vector.new(
  {
    California: 38332521, Texas: 26448193,
    NewYork: 19651127, Florida: 19552860,
    Illinois: 12882135
  }
)
dataframe = Daru::DataFrame.new({area:area, pop:pop})

#<Daru::DataFrame(5x2)>
                  area        pop
 California     423967   38332521
      Texas     695662   26448193
    NewYork     141297   19651127
    Florida     170312   19552860
   Illinois     149995   12882135

`DataFrame`に含まれる`Vector`には、bracket（`[]`）を使ってアクセスすることができます：

In [12]:
dataframe[:area]

#<Daru::Vector(5)>
 California     423967
      Texas     695662
    NewYork     141297
    Florida     170312
   Illinois     149995

列の名前が`String`や`Symbol`の場合は、メソッド呼び出しによってもアクセスすることができます：

In [13]:
dataframe.area

#<Daru::Vector(5)>
 California     423967
      Texas     695662
    NewYork     141297
    Florida     170312
   Illinois     149995

どちらの記法を使っても、アクセスされるオブジェクトは同一です：

In [14]:
dataframe.area.equal? dataframe[:area]

true

メソッド呼び出しは便利な機能ですが、動作しないこともあります。例えば、列の名前が`String`や`Symbol`でない場合や、列の名前が`DataFrame`のメソッドと衝突する場合などは動作しません。

bracketによって、オブジェクトを修正する事ができます：

In [15]:
dataframe[:density] = dataframe[:pop] / dataframe[:area]
dataframe

#<Daru::DataFrame(5x3)>
                  area        pop    density
 California     423967   38332521         90
      Texas     695662   26448193         38
    NewYork     141297   19651127        139
    Florida     170312   19552860        114
   Illinois     149995   12882135         85

### DataFrame as two-dimensional Array

別のアナロジーとして、`DataFrame`を二次元の`Array`と見ることもできます。`DataFrame`の内部にある生の`Array`は、`values`メソッドでアクセスできます：

In [ ]:
# dataframe.values

DataFrameに対して、多くの行列処理をすることができます。例えば、行と列を入れ替えた`DataFrame`は、以下のようにして取得することができます：

In [49]:
dataframe.transpose

#<Daru::DataFrame(3x5)>
            California      Texas    NewYork    Florida   Illinois
       area     423967     695662     141297     170312     149995
        pop   38332521   26448193   19651127   19552860   12882135
    density         90         38        139        114         85

In [54]:
# このへん何がいいたいのかわからない
# dataframe.values[0]

In [51]:
# このへん何がいいたいのかわからない
dataframe[:area]

#<Daru::Vector(5)>
 California     423967
      Texas     695662
    NewYork     141297
    Florida     170312
   Illinois     149995

次のように、`at`メソッドを使って、行と列のそれぞれの暗黙的インデックスを指定して、所望の範囲の値にアクセスすることもできます：

In [61]:
dataframe.at(0..1).row.at(0..2)

#<Daru::DataFrame(3x2)>
                  area        pop
 California     423967   38332521
      Texas     695662   26448193
    NewYork     141297   19651127

同様に、`hoge`メソッドを使って、行と列のそれぞれの明示的インデックスを指定して、所望の範囲の値にアクセスすることもできます：

（`loc`に相当するメソッドはない）

In [62]:
# dataframe.hoge(:area..:pop).row.hoge(:Texas..:Florida)

bracketを使えば、２つの記法をあわせたような方法で、所望の範囲の値にアクセスすることができます：

（`pandas`では、
```python
data.ix[:3, :'pop']
```
のようにアクセスするから意味があるけど、`Daru::DataFrame`だと、
```ruby
dataframe[:area..:pop].row.at(0..2)
```
みたいにアクセスすることになるので、「`[]`がexplicit indexとimplicit indexの両方を許容する」という特長が関係なくなる。）

In [69]:
dataframe[:area..:pop].row[0..2]

#<Daru::DataFrame(3x2)>
                  area        pop
 California     423967   38332521
      Texas     695662   26448193
    NewYork     141297   19651127

`where`と組み合わせる事もできます：

In [72]:
dataframe.where(dataframe.density.gt 100)[:pop, :density]

#<Daru::DataFrame(2x2)>
               pop  density
  NewYork 19651127      139
  Florida 19552860      114

どのアクセスの方法も、データの修正や指定に使うことができます：

（どのアクセスの方法でもできない）

In [86]:
# dataframe.at(0).at(2) = 90